In [1]:
pip install --upgrade pyod  

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install combo

In [3]:
import numpy as np 
import pandas as pd 

import os

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPRegressor

from scipy.stats.mstats import winsorize

In [5]:
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.lof import LOF

In [6]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install lightgbm 

Note: you may need to restart the kernel to use updated packages.


In [9]:
from xgboost import XGBRegressor, XGBRFRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import catboost
import lightgbm

In [10]:
#CSV to Load
FMR_21_data= "FY21_FMR.csv"
FMR21_df=pd.read_csv(FMR_21_data, na_values=['999.99'])
FMR21_df.head()
FMR21_df.tail()

,fips2010,fmr_0,fmr_1,fmr_2,fmr_3,fmr_4,state,metro_code,areaname,county,cousub,countyname,county_town_name,pop2017,state_alpha,metro
4761,7215199999,436,477,556,740,909,72,METRO41980MM7440,"San Juan-Guaynabo, PR HUD Metro FMR Area",151,99999,Yabucoa Municipio,Yabucoa Municipio,35025,PR,1
4762,7215399999,361,369,421,596,630,72,METRO38660M49500,"Yauco, PR HUD Metro FMR Area",153,99999,Yauco Municipio,Yauco Municipio,37585,PR,1
4763,7801099999,835,852,1040,1294,1410,78,NCNTY78010N78010,"St. Croix Island, VI",10,99999,St. Croix,St. Croix,53234,VI,0
4764,7802099999,1109,1314,1633,2031,2214,78,NCNTY78020N78020,"St. John Island, VI",20,99999,St. John,St. John,4197,VI,0
4765,7803099999,797,962,1204,1498,1633,78,NCNTY78030N78030,"St. Thomas Island, VI",30,99999,St. Thomas,St. Thomas,51181,VI,0


In [11]:
FMR21_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4766 entries, 0 to 4765
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   fips2010          4766 non-null   int64 
 1   fmr_0             4766 non-null   int64 
 2   fmr_1             4766 non-null   int64 
 3   fmr_2             4766 non-null   int64 
 4   fmr_3             4766 non-null   int64 
 5   fmr_4             4766 non-null   int64 
 6   state             4766 non-null   int64 
 7   metro_code        4766 non-null   object
 8   areaname          4766 non-null   object
 9   county            4766 non-null   int64 
 10  cousub            4766 non-null   int64 
 11  countyname        4766 non-null   object
 12  county_town_name  4766 non-null   object
 13  pop2017           4766 non-null   int64 
 14  state_alpha       4766 non-null   object
 15  metro             4766 non-null   int64 
dtypes: int64(11), object(5)
memory usage: 595.9+ KB


In [12]:
FMR21_df.isnull().sum()

fips2010            0
fmr_0               0
fmr_1               0
fmr_2               0
fmr_3               0
fmr_4               0
state               0
metro_code          0
areaname            0
county              0
cousub              0
countyname          0
county_town_name    0
pop2017             0
state_alpha         0
metro               0
dtype: int64

In [13]:
print("Categorical Columns:", list(FMR21_df.select_dtypes(include = 'object').columns))
print("Numerical Columns:", list(FMR21_df.select_dtypes(exclude = 'object').columns))

Categorical Columns: ['metro_code', 'areaname', 'countyname', 'county_town_name', 'state_alpha']
Numerical Columns: ['fips2010', 'fmr_0', 'fmr_1', 'fmr_2', 'fmr_3', 'fmr_4', 'state', 'county', 'cousub', 'pop2017', 'metro']


In [14]:
#

In [15]:
FMR21_df[FMR21_df['fmr_0']==0]['fmr_0'].count()

0

In [16]:
FMR21_studio_df=FMR21_df[FMR21_df['fmr_0']!=0]
FMR21_studio_df.head()

,fips2010,fmr_0,fmr_1,fmr_2,fmr_3,fmr_4,state,metro_code,areaname,county,cousub,countyname,county_town_name,pop2017,state_alpha,metro
0,100199999,640,766,908,1148,1520,1,METRO33860M33860,"Montgomery, AL MSA",1,99999,Autauga County,Autauga County,55035,AL,1
1,100399999,718,723,922,1249,1584,1,METRO19300M19300,"Daphne-Fairhope-Foley, AL MSA",3,99999,Baldwin County,Baldwin County,203360,AL,1
2,100599999,488,492,648,806,907,1,NCNTY01005N01005,"Barbour County, AL",5,99999,Barbour County,Barbour County,26200,AL,0
3,100799999,817,871,1002,1303,1409,1,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",7,99999,Bibb County,Bibb County,22580,AL,1
4,100999999,817,871,1002,1303,1409,1,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",9,99999,Blount County,Blount County,57665,AL,1


In [17]:
CFMR21_df = FMR21_df.drop(['state','metro_code', 'fips2010','areaname','county_town_name','metro', 'cousub','county'], axis=1)
CFMR21_df.head()

,fmr_0,fmr_1,fmr_2,fmr_3,fmr_4,countyname,pop2017,state_alpha
0,640,766,908,1148,1520,Autauga County,55035,AL
1,718,723,922,1249,1584,Baldwin County,203360,AL
2,488,492,648,806,907,Barbour County,26200,AL
3,817,871,1002,1303,1409,Bibb County,22580,AL
4,817,871,1002,1303,1409,Blount County,57665,AL


In [18]:
CFMR21_df.rename(columns = {'fmr_0':'Studio', 'fmr_1':'One Bedroom', 'fmr_2':'Two Bedroom', 'fmr_3':'Three Bedroom','fmr_4':'Four Bedroom','pop2017':'Population','countyname':'County','state_alpha':'State'}, inplace = True)

In [19]:
CFMR21_df=CFMR21_df.groupby(by=['State'], dropna=False).mean()

In [20]:
CFMR21_df.insert(0, 'Year', '2021')

In [21]:
CFMR21_df=CFMR21_df.round(2)

In [23]:
CFMR21_df.head()

,Year,Studio,One Bedroom,Two Bedroom,Three Bedroom,Four Bedroom,Population
State,,,,,,,
AK,2021,850.97,951.28,1200.14,1582.72,1815.38,25467.93
AL,2021,548.28,591.99,725.82,949.51,1094.63,72399.63
AR,2021,515.95,553.85,697.31,920.29,1065.15,39706.00
AS,2021,590.00,594.00,783.00,1121.00,1201.00,55519.00
AZ,2021,698.13,767.93,970.13,1340.20,1493.53,453995.67


In [24]:
CFMR21_df.to_csv(r"CFMR21_df.csv")

In [ ]:
numeric_columns = list(FMR21_df.select_dtypes(exclude = 'object').columns)
fig = make_subplots(rows=2, cols=3, shared_yaxes=True)
index = 0
for row in range(1,3):
    for col in range(1,4):
        fig.add_trace(go.Box(x=FMR21_df[numeric_columns[index]], name = numeric_columns[index]),
                      row = row,
                      col = col,
)
        index = index + 1
fig.update_layout(height=600, width=1300,
                  title_text="Box-Plots of Numerical Variables")

fig.show()

In [ ]:
sns.regplot(x="Studio", y="Population", data=FMR21_df)
plt.ylim(0,)